In [245]:
import pandas as pd

In [246]:
train_path = "./train.csv"
test_path = "./test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [247]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [248]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing
it contains:
- dropping passengerID, name, fare, cabin, ticket
- changing sex & embarked into numerical
- scaling

In [249]:
# use train dataset to find this
def find_filler(df):
    # find the average age per class
    average_age_class = df.groupby('Pclass')['Age'].mean()
    # find the mode embarked per class
    mode_embarked_class = df.groupby('Pclass')['Embarked'].agg(lambda x: x.mode().iloc[0])
    
    average_age_class = list(average_age_class)
    mode_embarked_class = list(mode_embarked_class)
    return average_age_class+mode_embarked_class

# shape probably like this [age1, age2, age3, mode1, mode2, mode3]
filler = find_filler(train_df)
print(filler)

[38.233440860215055, 29.87763005780347, 25.14061971830986, 'S', 'S', 'S']


In [250]:
pd.set_option('display.max_rows', None)

In [251]:
# dummy_df = test_df.copy()
# nan value is filled based on the Pclass, hence why the long ahh code
# fill each age and embarked null (connected to fill_na function)
def fill(df, age, column):
  df.loc[df["Age"].isna(), "Age"] = age
  df.loc[df["Embarked"].isna(), "Embarked"] = column
  return df

# fill the nan value
def fill_na(df, prep_type, values):
    # separate df based on Pclass
    df1 = df.loc[df["Pclass"]==1]
    df2 = df.loc[df["Pclass"]==2]
    df3 = df.loc[df["Pclass"]==3]
    df_list = [df1, df2, df3]
    # fill the nan value in df1, df2, df33
    for idx, x in enumerate(df_list):
        # print(df_list[idx].head())
        df_list[idx] = fill(x, int(values[idx]), values[3+idx])

    # concat all the separated df
    final_df = pd.concat([df_list[0], df_list[1], df_list[2]])
    # if train df, shuffle the dataset
    if prep_type == "Train":
        final_df = final_df.sample(frac=1, random_state=64)
    # if test df, sort the dataset based on passenger id (to match the correct answer)
    else:
        final_df.sort_values(by=["PassengerId"], inplace=True)
    
    return final_df
    

# preprocessing
def preprocess(df, type, nan_filler):
    # in test we dont drop the passenger id to sort the answer
    if type == "Train":
        # drop passenger id
        df.drop("PassengerId", axis=1, inplace=True)

    # dropping name, fare, cabin, ticket
    df.drop(columns=["Name", "Fare", "Cabin", "Ticket"], inplace=True)
    
    # changing sex  into numerical
    df["Sex"].replace("male", 1, inplace=True)
    df["Sex"].replace("female", 0, inplace=True)

    # input fillna here
    df = fill_na(df, type, nan_filler)

    # change embarked into numerical
    df["Embarked"].replace('C', 1, inplace=True)
    df["Embarked"].replace('Q', 2, inplace=True)
    df["Embarked"].replace('S', 3, inplace=True)

    # split x and y
    # case: 
    # Train: x -> feature, y -> survived
    # Test: x -> feature, y -> PassengerID
    labels = list(df.columns)
    x = df[labels[1:]]
    y = df[labels[0]]

    return x, y

In [252]:
x_train, y_train = preprocess(train_df, "Train", filler)
x_test, test_PassengerId = preprocess(test_df, "Test", filler)

In [253]:
# scaling with standard scaler (can use min max too)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [254]:
# # fix imbalance with oversampling
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42)
# x_train, y_train = sm.fit_resample(x_train, y_train)

In [255]:
# # how about PCA? (it reduces the accuracy)
# from sklearn.decomposition import PCA
# pca = PCA(n_components=4)
# pca.fit(x_train)
# x_train = pca.transform(x_train)
# x_test = pca.transform(x_test)

In [256]:
# Model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

Survived = model.predict(x_test)

In [257]:
passenger_id = list(test_PassengerId)
Survived = list(Survived)
datas = {
    "PassengerId" : passenger_id,
    "Survived" : Survived
}
ans = pd.DataFrame(datas)
ans.head() 

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [258]:
ans.to_csv("prediction.csv", index=False)
print("Test zz")

Test zz
